In [5]:
import pandas as pd
import numpy as np
import math

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
qna_df = pd.read_excel('/content/drive/MyDrive/AI_proj/qna_cleaned.xlsx',sheet_name = 'qna_cleaned')
multi_qna_df = pd.read_excel('/content/drive/MyDrive/AI_proj/multiple_qna_cleaned.xlsx',sheet_name = 'multiple_qna_cleaned')

In [20]:
qna_df.head()

,Unnamed: 0,id,article_id,menu_id,question,answer,q_mbti,a_mbti
0,0,1,804077,11,궁금해요 [SEP],공방 가서 맞추는 건 싫고 금으로 맞추는 거 아님[SEP]싫어요 ㅎ[SEP],NaN,istj
1,1,2,804070,11,사귄 지 얼마 안 됐고 섬 탈 때도 그냥 용건만 연락하는 편이길래 그러려니 했는데 ...,원하시는 일상 공유가 어떤 건지 [SEP]궁금해요 [SEP],NaN,intp
2,2,4,804044,11,알려줘야 된다고 생각하세요?[SEP]아니면 굳이 필요 없다고 생각하세요?[SEP]자...,저는 무조건 ..![SEP]말해야 된다고 생각해요 [SEP],NaN,enfp
3,3,5,804033,11,애칭 어떻게 부르셨는지 알려주세요[SEP],자기 정도만 쓰는 거 같아요.[SEP] 애인을 지칭할 때 '너'라고 하기에는 너무 ...,intj,istj
4,4,6,804012,11,나만 남자친구랑 데이트 생각하는 거 같고 어찌나 서운하던지 근데 또 집에 가면 편해...,직장 상사 스타일이라고 생각하심 돼요 [SEP]제가 느끼기에는 f 부분이 너무 적어...,infp,estp


In [21]:
multi_qna_df.head()

,Unnamed: 0,id,article_id,menu_id,question,answer,q_mbti,a_mbti
0,0,1,806189,72,당첨되게 해주세요[SEP]비나이다 비나이다~[SEP] 문의는 dm으로 부탁드릴게요 ...,전 가족한테도 알리지 않을 생각이에요.[SEP]그 돈으로 나태해질 사람들 보고 싶지...,infp,infj
1,1,2,806189,72,당첨되게 해주세요[SEP]비나이다 비나이다~[SEP] 문의는 dm으로 부탁드릴게요 ...,저는 연인이랑 엄마한테만 알릴 거예요![SEP],infp,infp
2,2,3,806189,72,당첨되게 해주세요[SEP]비나이다 비나이다~[SEP] 문의는 dm으로 부탁드릴게요 ...,가족한테 안 알리고 남자친구가 있다면 알리지 않았다가 결혼할 때 딱 말해주고 싶어요...,infp,enfj
3,3,6,806189,72,당첨되게 해주세요[SEP]비나이다 비나이다~[SEP] 문의는 dm으로 부탁드릴게요 ...,"부모님께만 살짝 알릴 거예요, 좋은 집 사면 주변에서 알게 되겠지만 스스로 일찍은 ...",infp,isfj
4,4,7,806189,72,당첨되게 해주세요[SEP]비나이다 비나이다~[SEP] 문의는 dm으로 부탁드릴게요 ...,가족들한테 바로 알려서 기쁨을 같이 나누고 같이 나눠 쓸거예요ㅎㅎ[SEP],infp,infp


In [22]:
# unnamed라는 불필요한 칼럼 삭제 및 id등 트레인 데이터에 불필요한 칼럼 삭제

multi_qna_df = multi_qna_df.drop(['Unnamed: 0', 'id', 'article_id', 'menu_id'], axis = 1)
qna_df = qna_df.drop(['Unnamed: 0', 'id', 'article_id', 'menu_id'], axis = 1)

In [23]:
multi_qna_df.head()

,question,answer,q_mbti,a_mbti
0,당첨되게 해주세요[SEP]비나이다 비나이다~[SEP] 문의는 dm으로 부탁드릴게요 ...,전 가족한테도 알리지 않을 생각이에요.[SEP]그 돈으로 나태해질 사람들 보고 싶지...,infp,infj
1,당첨되게 해주세요[SEP]비나이다 비나이다~[SEP] 문의는 dm으로 부탁드릴게요 ...,저는 연인이랑 엄마한테만 알릴 거예요![SEP],infp,infp
2,당첨되게 해주세요[SEP]비나이다 비나이다~[SEP] 문의는 dm으로 부탁드릴게요 ...,가족한테 안 알리고 남자친구가 있다면 알리지 않았다가 결혼할 때 딱 말해주고 싶어요...,infp,enfj
3,당첨되게 해주세요[SEP]비나이다 비나이다~[SEP] 문의는 dm으로 부탁드릴게요 ...,"부모님께만 살짝 알릴 거예요, 좋은 집 사면 주변에서 알게 되겠지만 스스로 일찍은 ...",infp,isfj
4,당첨되게 해주세요[SEP]비나이다 비나이다~[SEP] 문의는 dm으로 부탁드릴게요 ...,가족들한테 바로 알려서 기쁨을 같이 나누고 같이 나눠 쓸거예요ㅎㅎ[SEP],infp,infp


In [24]:
qna_df.head()

,question,answer,q_mbti,a_mbti
0,궁금해요 [SEP],공방 가서 맞추는 건 싫고 금으로 맞추는 거 아님[SEP]싫어요 ㅎ[SEP],NaN,istj
1,사귄 지 얼마 안 됐고 섬 탈 때도 그냥 용건만 연락하는 편이길래 그러려니 했는데 ...,원하시는 일상 공유가 어떤 건지 [SEP]궁금해요 [SEP],NaN,intp
2,알려줘야 된다고 생각하세요?[SEP]아니면 굳이 필요 없다고 생각하세요?[SEP]자...,저는 무조건 ..![SEP]말해야 된다고 생각해요 [SEP],NaN,enfp
3,애칭 어떻게 부르셨는지 알려주세요[SEP],자기 정도만 쓰는 거 같아요.[SEP] 애인을 지칭할 때 '너'라고 하기에는 너무 ...,intj,istj
4,나만 남자친구랑 데이트 생각하는 거 같고 어찌나 서운하던지 근데 또 집에 가면 편해...,직장 상사 스타일이라고 생각하심 돼요 [SEP]제가 느끼기에는 f 부분이 너무 적어...,infp,estp


In [25]:
# [SEP]문자 제거

qna_df['question'] = qna_df['question'].str.replace("SEP", ' ').str.replace('[', '').str.replace(']','')
qna_df['answer'] = qna_df['answer'].str.replace("SEP", ' ').str.replace('[', '').str.replace(']','')
multi_qna_df['question'] = multi_qna_df['question'].str.replace("SEP", ' ').str.replace('[', '').str.replace(']','')
multi_qna_df['answer'] = multi_qna_df['answer'].str.replace("SEP", ' ').str.replace('[', '').str.replace(']','')

<ipython-input-25-80f17ac2672b>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  qna_df['question'] = qna_df['question'].str.replace("SEP", ' ').str.replace('[', '').str.replace(']','')
<ipython-input-25-80f17ac2672b>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  qna_df['answer'] = qna_df['answer'].str.replace("SEP", ' ').str.replace('[', '').str.replace(']','')
<ipython-input-25-80f17ac2672b>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  multi_qna_df['question'] = multi_qna_df['question'].str.replace("SEP", ' ').s

In [26]:
qna_df.head()

,question,answer,q_mbti,a_mbti
0,궁금해요,공방 가서 맞추는 건 싫고 금으로 맞추는 거 아님 싫어요 ㅎ,NaN,istj
1,사귄 지 얼마 안 됐고 섬 탈 때도 그냥 용건만 연락하는 편이길래 그러려니 했는데 ...,원하시는 일상 공유가 어떤 건지 궁금해요,NaN,intp
2,알려줘야 된다고 생각하세요? 아니면 굳이 필요 없다고 생각하세요? 자잘한 일상 공유...,저는 무조건 ..! 말해야 된다고 생각해요,NaN,enfp
3,애칭 어떻게 부르셨는지 알려주세요,자기 정도만 쓰는 거 같아요. 애인을 지칭할 때 '너'라고 하기에는 너무 정 없어...,intj,istj
4,나만 남자친구랑 데이트 생각하는 거 같고 어찌나 서운하던지 근데 또 집에 가면 편해...,직장 상사 스타일이라고 생각하심 돼요 제가 느끼기에는 f 부분이 너무 적어요..,infp,estp


In [27]:
multi_qna_df.head()

,question,answer,q_mbti,a_mbti
0,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,전 가족한테도 알리지 않을 생각이에요. 그 돈으로 나태해질 사람들 보고 싶지 않아서요,infp,infj
1,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,저는 연인이랑 엄마한테만 알릴 거예요!,infp,infp
2,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,가족한테 안 알리고 남자친구가 있다면 알리지 않았다가 결혼할 때 딱 말해주고 싶어요...,infp,enfj
3,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,"부모님께만 살짝 알릴 거예요, 좋은 집 사면 주변에서 알게 되겠지만 스스로 일찍은 ...",infp,isfj
4,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,가족들한테 바로 알려서 기쁨을 같이 나누고 같이 나눠 쓸거예요ㅎㅎ,infp,infp


In [31]:
# MBTI가 NaN인 행 삭제

qna_df_na = qna_df.dropna(axis = 0)
multi_qna_df_na = multi_qna_df.dropna(axis = 0)

In [64]:
qna_df_na = qna_df_na.reset_index(drop = True)
multi_qna_df_na = multi_qna_df_na.reset_index(drop = True)

In [66]:
qna_df_na.head()

,question,answer,q_mbti,a_mbti
0,애칭 어떻게 부르셨는지 알려주세요,자기 정도만 쓰는 거 같아요. 애인을 지칭할 때 '너'라고 하기에는 너무 정 없어...,intj,istj
1,나만 남자친구랑 데이트 생각하는 거 같고 어찌나 서운하던지 근데 또 집에 가면 편해...,직장 상사 스타일이라고 생각하심 돼요 제가 느끼기에는 f 부분이 너무 적어요..,infp,estp
2,istj와 isfj는 알파벳 한 끗 차이인데 istj이시면서 t와 f가 거의 비슷한...,저도 f/t 왔다 갔다 해서 궁금하네요.!! 오지랖 넓은 istj는 없겠죠..,entp,istj
3,잇디 제분들 직장에서 수다떨기 좋아하시나요?,상대가 누군지에 따라 다르겠죠??,isfj,istj
4,estj 남 entj여 인 분들 연애 어떻게 하시고 계시나요? 사귄 지 얼마 안 됐...,남자친구가 에스티 제고 제가 entj입니다 ㅋㅋㅋ 300일 넘게 사귀고 결혼 준비 ...,entj,entj


In [67]:
multi_qna_df_na.head()

,question,answer,q_mbti,a_mbti
0,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,전 가족한테도 알리지 않을 생각이에요. 그 돈으로 나태해질 사람들 보고 싶지 않아서요,infp,infj
1,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,저는 연인이랑 엄마한테만 알릴 거예요!,infp,infp
2,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,가족한테 안 알리고 남자친구가 있다면 알리지 않았다가 결혼할 때 딱 말해주고 싶어요...,infp,enfj
3,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,"부모님께만 살짝 알릴 거예요, 좋은 집 사면 주변에서 알게 되겠지만 스스로 일찍은 ...",infp,isfj
4,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,가족들한테 바로 알려서 기쁨을 같이 나누고 같이 나눠 쓸거예요ㅎㅎ,infp,infp


In [68]:
len(qna_df)

31066

In [69]:
len(qna_df_na)

22300

In [70]:
len(multi_qna_df)

88417

In [71]:
len(multi_qna_df_na)

61088

In [98]:
# MBTI가 통합된 데이터 csv파일 생성

qna_df_na.to_csv('/content/drive/MyDrive/AI_proj/qna_cleaned_na.csv', index=False)
multi_qna_df_na.to_csv('/content/drive/MyDrive/AI_proj/multiple_qna_cleaned_na.csv', index=False)

In [102]:
# 합치기

df_qna_na = pd.concat([qna_df_na, multi_qna_df_na], ignore_index=True)
df_qna_na.to_csv('/content/drive/MyDrive/AI_proj/df_qna_cleaned_na.csv', index=False)

In [89]:
# MBTI칼럼을 4개로 나눠보기

qna_df_na_4 = pd.DataFrame(qna_df_na)

qna_df_na_4["q_mbti_ie"] = qna_df_na_4["q_mbti"].str[0]
qna_df_na_4["q_mbti_ns"] = qna_df_na_4["q_mbti"].str[1]
qna_df_na_4["q_mbti_tf"] = qna_df_na_4["q_mbti"].str[2]
qna_df_na_4["q_mbti_pj"] = qna_df_na_4["q_mbti"].str[3]

qna_df_na_4["a_mbti_ie"] = qna_df_na_4["a_mbti"].str[0]
qna_df_na_4["a_mbti_ns"] = qna_df_na_4["a_mbti"].str[1]
qna_df_na_4["a_mbti_tf"] = qna_df_na_4["a_mbti"].str[2]
qna_df_na_4["a_mbti_pj"] = qna_df_na_4["a_mbti"].str[3]


multi_qna_df_na_4 = pd.DataFrame(multi_qna_df_na)

multi_qna_df_na_4["q_mbti_ie"] = multi_qna_df_na_4["q_mbti"].str[0]
multi_qna_df_na_4["q_mbti_ns"] = multi_qna_df_na_4["q_mbti"].str[1]
multi_qna_df_na_4["q_mbti_tf"] = multi_qna_df_na_4["q_mbti"].str[2]
multi_qna_df_na_4["q_mbti_pj"] = multi_qna_df_na_4["q_mbti"].str[3]

multi_qna_df_na_4["a_mbti_ie"] = multi_qna_df_na_4["a_mbti"].str[0]
multi_qna_df_na_4["a_mbti_ns"] = multi_qna_df_na_4["a_mbti"].str[1]
multi_qna_df_na_4["a_mbti_tf"] = multi_qna_df_na_4["a_mbti"].str[2]
multi_qna_df_na_4["a_mbti_pj"] = multi_qna_df_na_4["a_mbti"].str[3]

In [91]:
qna_df_na_4.head()

,question,answer,q_mbti,a_mbti,q_mbti_ie,q_mbti_ns,q_mbti_tf,q_mbti_pj,a_mbti_ie,a_mbti_ns,a_mbti_tf,a_mbti_pj
0,애칭 어떻게 부르셨는지 알려주세요,자기 정도만 쓰는 거 같아요. 애인을 지칭할 때 '너'라고 하기에는 너무 정 없어...,intj,istj,i,n,t,j,i,s,t,j
1,나만 남자친구랑 데이트 생각하는 거 같고 어찌나 서운하던지 근데 또 집에 가면 편해...,직장 상사 스타일이라고 생각하심 돼요 제가 느끼기에는 f 부분이 너무 적어요..,infp,estp,i,n,f,p,e,s,t,p
2,istj와 isfj는 알파벳 한 끗 차이인데 istj이시면서 t와 f가 거의 비슷한...,저도 f/t 왔다 갔다 해서 궁금하네요.!! 오지랖 넓은 istj는 없겠죠..,entp,istj,e,n,t,p,i,s,t,j
3,잇디 제분들 직장에서 수다떨기 좋아하시나요?,상대가 누군지에 따라 다르겠죠??,isfj,istj,i,s,f,j,i,s,t,j
4,estj 남 entj여 인 분들 연애 어떻게 하시고 계시나요? 사귄 지 얼마 안 됐...,남자친구가 에스티 제고 제가 entj입니다 ㅋㅋㅋ 300일 넘게 사귀고 결혼 준비 ...,entj,entj,e,n,t,j,e,n,t,j


In [90]:
multi_qna_df_na_4.head()

,question,answer,q_mbti,a_mbti,q_mbti_ie,q_mbti_ns,q_mbti_tf,q_mbti_pj,a_mbti_ie,a_mbti_ns,a_mbti_tf,a_mbti_pj
0,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,전 가족한테도 알리지 않을 생각이에요. 그 돈으로 나태해질 사람들 보고 싶지 않아서요,infp,infj,i,n,f,p,i,n,f,j
1,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,저는 연인이랑 엄마한테만 알릴 거예요!,infp,infp,i,n,f,p,i,n,f,p
2,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,가족한테 안 알리고 남자친구가 있다면 알리지 않았다가 결혼할 때 딱 말해주고 싶어요...,infp,enfj,i,n,f,p,e,n,f,j
3,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,"부모님께만 살짝 알릴 거예요, 좋은 집 사면 주변에서 알게 되겠지만 스스로 일찍은 ...",infp,isfj,i,n,f,p,i,s,f,j
4,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,가족들한테 바로 알려서 기쁨을 같이 나누고 같이 나눠 쓸거예요ㅎㅎ,infp,infp,i,n,f,p,i,n,f,p


In [92]:
# MBTI가 분리된 데이터 csv파일 생성

qna_df_na_4.to_csv('/content/drive/MyDrive/AI_proj/qna_cleaned_na_4.csv', index=False)
multi_qna_df_na_4.to_csv('/content/drive/MyDrive/AI_proj/multiple_qna_cleaned_na_4.csv', index=False)

In [103]:
# 합치기

df_qna_na_4 = pd.concat([qna_df_na_4, multi_qna_df_na_4], ignore_index=True)
df_qna_na_4.to_csv('/content/drive/MyDrive/AI_proj/df_qna_cleaned_na_4.csv', index=False)

In [115]:
# question 이랑 answer 랑 별개의 행으로

a = qna_df_na[['question', 'q_mbti']]
b = qna_df_na[['answer', 'a_mbti']]

a.columns = ['text', 'mbti']
b.columns = ['text', 'mbti']

qna_cleaned_na_text = pd.concat([a, b], ignore_index=True)
qna_cleaned_na_text.head()


m_a = multi_qna_df_na[['question', 'q_mbti']]
m_b = multi_qna_df_na[['answer', 'a_mbti']]

m_a.columns = ['text', 'mbti']
m_b.columns = ['text', 'mbti']

multi_qna_cleaned_na_text = pd.concat([m_a, m_b], ignore_index=True)
multi_qna_cleaned_na_text.head()

,text,mbti
0,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,infp
1,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,infp
2,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,infp
3,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,infp
4,당첨되게 해주세요 비나이다 비나이다~ 문의는 dm으로 부탁드릴게요,infp


In [116]:
len(qna_cleaned_na_text)

44600

In [117]:
len(multi_qna_cleaned_na_text)

122176

In [120]:
# 중복 행 제거

multi_qna_cleaned_na_text = multi_qna_cleaned_na_text.drop_duplicates()

In [126]:
len(multi_qna_cleaned_na_text)

77480

In [121]:
qna_cleaned_na_text = qna_cleaned_na_text.reset_index(drop = True)
multi_qna_cleaned_na_text = multi_qna_cleaned_na_text.reset_index(drop = True)

In [124]:
# MBTI가 통합된 데이터 csv파일 생성

qna_cleaned_na_text.to_csv('/content/drive/MyDrive/AI_proj/qna_cleaned_na_text.csv', index=False)
multi_qna_cleaned_na_text.to_csv('/content/drive/MyDrive/AI_proj/multiple_qna_cleaned_na_text.csv', index=False)

In [123]:
# 합치기

df_text_na = pd.concat([qna_cleaned_na_text, multi_qna_cleaned_na_text], ignore_index=True)
df_text_na.to_csv('/content/drive/MyDrive/AI_proj/df_text_cleaned_na.csv', index=False)

In [127]:
len(df_text_na)

122080

In [125]:
df_text_na.head()

,text,mbti
0,애칭 어떻게 부르셨는지 알려주세요,intj
1,나만 남자친구랑 데이트 생각하는 거 같고 어찌나 서운하던지 근데 또 집에 가면 편해...,infp
2,istj와 isfj는 알파벳 한 끗 차이인데 istj이시면서 t와 f가 거의 비슷한...,entp
3,잇디 제분들 직장에서 수다떨기 좋아하시나요?,isfj
4,estj 남 entj여 인 분들 연애 어떻게 하시고 계시나요? 사귄 지 얼마 안 됐...,entj


In [129]:
# MBTI칼럼을 4개로 나눠보기

df_text_na_4 = pd.DataFrame(df_text_na)

df_text_na_4["mbti_ie"] = df_text_na_4["mbti"].str[0]
df_text_na_4["mbti_ns"] = df_text_na_4["mbti"].str[1]
df_text_na_4["mbti_tf"] = df_text_na_4["mbti"].str[2]
df_text_na_4["mbti_pj"] = df_text_na_4["mbti"].str[3]

In [130]:
df_text_na_4.head()

,text,mbti,mbti_ie,mbti_ns,mbti_tf,mbti_pj
0,애칭 어떻게 부르셨는지 알려주세요,intj,i,n,t,j
1,나만 남자친구랑 데이트 생각하는 거 같고 어찌나 서운하던지 근데 또 집에 가면 편해...,infp,i,n,f,p
2,istj와 isfj는 알파벳 한 끗 차이인데 istj이시면서 t와 f가 거의 비슷한...,entp,e,n,t,p
3,잇디 제분들 직장에서 수다떨기 좋아하시나요?,isfj,i,s,f,j
4,estj 남 entj여 인 분들 연애 어떻게 하시고 계시나요? 사귄 지 얼마 안 됐...,entj,e,n,t,j


In [131]:
df_text_na_4.to_csv('/content/drive/MyDrive/AI_proj/df_text_cleaned_na_4.csv', index=False)